In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import TimeSeriesSplit
from scipy import signal
import glob

In [59]:
csv_files = glob.glob(
    r"C:\Users\Max_G\OneDrive\IUBH\5_Semester\Model_Engeneering\Public_Transport_Forecasting\data\interim\Cluster_Data_All_Month\*.csv"
)

In [60]:
weekday_file_paths = [path for path in csv_files if 'weekday' in path]
weekend_file_paths = [path for path in csv_files if 'weekend' in path]

In [61]:
weekday_dict = {}

for index, file in enumerate(weekday_file_paths):
    current_df = pd.read_csv(file, index_col=[0])
    current_df['Date'] =  pd.to_datetime(current_df['Date'])
    current_df.index = pd.to_datetime(current_df.index)

    weekday_dict[int(weekday_file_paths[index][-13:-12])] = current_df

In [62]:
weekend_dict = {}  

for index, file in enumerate(weekend_file_paths):
    current_df = pd.read_csv(file, index_col=[0])
    current_df['Date'] = pd.to_datetime(current_df['Date'])
    current_df.index = pd.to_datetime(current_df.index)

    weekend_dict[int(weekend_file_paths[index][-13:-12])] = current_df

In [63]:
endog_week_dict = {}

for label in weekday_dict:
    current_df = weekday_dict[label]
    current_df = current_df.reset_index()
    current_df = current_df.drop(columns=['index', 'Hour', 'Weekday', 'Date'])
    endog_week_dict[label] = current_df

In [64]:
endog_weekend_dict = {}

for label in weekend_dict:
    current_df = weekend_dict[label]
    current_df = current_df.reset_index()
    current_df = current_df.drop(columns=['index', 'Hour', 'Weekday', 'Date'])
    endog_weekend_dict[label] = current_df

In [65]:
north_manhattan_week = pd.read_csv(r"C:\Users\Max_G\OneDrive\IUBH\5_Semester\Model_Engeneering\Public_Transport_Forecasting\data\interim\Cluster_Data_All_Month\df_0_weekday.csv", index_col=[0])
north_manhattan_week['Date'] = pd.to_datetime(north_manhattan_week['Date'])
north_manhattan_week.index = pd.to_datetime(north_manhattan_week.index)


endog_north_manhattan_week = north_manhattan_week.reset_index()
endog_north_manhattan_week = endog_north_manhattan_week.drop(columns=['index', 'Hour', 'Weekday', 'Date'])


In [66]:
def getMaxPeriodogram(freq, Pxx_spec, num_top=5):
    # Find the indices of the top 'num_top' maximum PSDs
    top_indices = np.argsort(Pxx_spec)[-num_top:][::-1]

    # Extract the frequencies and PSDs for the top 'num_top' maximum PSDs
    freqMax = freq[top_indices]
    maxPsd = Pxx_spec[top_indices]
    # Convert to periods
    periodMax = 1 / freqMax
    return maxPsd, freqMax, periodMax

In [209]:
def perform_cross_validation(endog_data, n_splits):
    # Number of splits for cross-validation
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Lists to store Mean Absolute Error (MAE) values
    mae_list_train_sarima = []
    mae_list_test_sarima = []
    mae_list_aic_sarima = []
    mae_list_train_sarimax = []
    mae_list_test_sarimax = []
    mae_list_aic_sarimax = []

    a = 0

    

    # Iterate through cross-validation splits
    for train_index, test_index in tscv.split(endog_data["Count"]):
        if len(train_index) >= len(endog_data) / 2:
            train_data = endog_data["Count"][train_index]
            test_data = endog_data["Count"][test_index]

            # Fit SARIMA model on the training data
            order = (0, 1, 6)
            seasonal_order = (1, 1, 1, 24)

            sarima_model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order)
            sarima_fit = sarima_model.fit()
            mae_sarima_train = np.mean(np.abs(train_data.values - sarima_fit.fittedvalues))
            print(a)
            a += 1
            mae_list_train_sarima.append(mae_sarima_train)
            mae_list_aic_sarima.append(sarima_fit.aic)

            # Forecast using the fitted model
            forecast = sarima_fit.forecast(steps=len(test_data))

            # Evaluate forecast against actual values
            mae_sarima_test = np.mean(np.abs(test_data.values - forecast.values))
            mae_list_test_sarima.append(mae_sarima_test)

            # Generate periodogram
            freq, Pxx_spec = signal.periodogram(
                endog_data["Count"][
                    : len(np.concatenate((train_index, test_index)))
                ].values,
                scaling="spectrum",
            )

            maxPsd, freqMax_week, periodMax = getMaxPeriodogram(freq, Pxx_spec, 5)

            frequency_array = np.round(1 / freqMax_week).astype(int)
            period01, period02, period03, period04, period05 = frequency_array

            time = np.arange(0, len(np.concatenate((train_index, test_index))))

            # Create regressors
            omega01 = 2 * np.pi / period01
            s1 = np.cos(omega01 * time)
            c1 = np.sin(omega01 * time)

            # Create regressors
            omega02 = 2 * np.pi / period02
            s2 = np.cos(omega02 * time)
            c2 = np.sin(omega02 * time)

            # Create regressors
            omega03 = 2 * np.pi / period03
            s3 = np.cos(omega03 * time)
            c3 = np.sin(omega03 * time)

            # Create regressors
            omega04 = 2 * np.pi / period04
            s4 = np.cos(omega04 * time)
            c4 = np.sin(omega04 * time)

            # Create regressors
            omega05 = 2 * np.pi / period05
            s5 = np.cos(omega05 * time)
            c5 = np.sin(omega05 * time)

            order = (24, 0, 0)

            regressors = np.column_stack((s1, c1, s2, c2, s3, c3, s4, c4, s5, c5))
            try:
                model = SARIMAX(
                    endog=train_data,
                    exog=regressors[: -len(test_index)],
                    trend="c",
                    order=order,
                    enforce_stationarity=True,
                    enforce_invertibility=True,
                )
                sarimax_model = model.fit()
                mae_list_aic_sarimax.append(sarimax_model.aic)
                mae_sarimax_train = np.mean(
                    np.abs(train_data.values - sarimax_model.fittedvalues)
                )
                mae_list_train_sarimax.append(mae_sarimax_train)

                # Forecasting
                forecast = sarimax_model.forecast(
                    steps=len(test_data), exog=regressors[-len(test_index) :]
                )
                # Evaluate forecast against actual values
                mae_sarimax_test = np.mean(
                    np.abs(test_data.values - forecast.values)
                )
                mae_list_test_sarimax.append(mae_sarimax_test)
            except np.linalg.LinAlgError as linalg_err:
                # Handle the LinAlgError here
                print("Caught a LinAlgError as an exception:", linalg_err)
                mae_list_test_sarimax.append("error")
                mae_list_aic_sarimax.append("error")
                mae_list_train_sarimax.append("error")

    df_testing = pd.DataFrame({
        'aic_sarima': mae_list_aic_sarima,
        'mae_test_sarima': mae_list_test_sarima,
        'mae_train_sarima': mae_list_train_sarima,
        'mae_train_sarimax': mae_list_train_sarimax,
        'mae_test_sarimax': mae_list_test_sarimax,
        'aic_sarimax': mae_list_aic_sarimax,

    })
    
    return df_testing


In [197]:
all_results = []

# Iterate through labels in endog_week_dict
for label, endog_data in endog_week_dict.items():
    print('this is the current label', label)
    results = perform_cross_validation(endog_data, 20)
    results['label'] = label
    all_results.append(results)

# Concatenate all DataFrames into one
df_combined = pd.concat(all_results, ignore_index=True)

folder_path = r"C:\Users\Max_G\OneDrive\IUBH\5_Semester\Model_Engeneering\Public_Transport_Forecasting\data\interim\Experiments_Results"
file_name = "df_mae_all_data_all_models_week.csv"

full_path = f"{folder_path}/{file_name}"

df_combined.to_csv(full_path, index=True)





this is the current label 0
0
453.28731603566973
157.40432631500457
1
91.88728594031477
123.40479666915454
2
88.57051619590342


KeyboardInterrupt: 

In [211]:
def train_test_split(endog_data, testing_lenght, splits):
    # Lists to store Mean Absolute Error (MAE) values
    mae_list_train_sarima = []
    mae_list_test_sarima = []
    mae_list_aic_sarima = []
    mae_list_train_sarimax = []
    mae_list_test_sarimax = []
    mae_list_aic_sarimax = []

    a = 0

    for repeats in range(0, splits):
        initial_point = (len(endog_data) // testing_lenght) - splits
        train_index = initial_point * testing_lenght + repeats * testing_lenght
        train_data = endog_data[:train_index]
        test_index = train_index + testing_lenght
        test_data = endog_data[train_index:test_index]

         # Fit SARIMA model on the training data
        order = (0, 1, 6)
        seasonal_order = (1, 1, 1, 24)

        sarima_model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order)
        sarima_fit = sarima_model.fit()
        mae_sarima_train = np.mean(np.abs(train_data['Count'] - sarima_fit.fittedvalues))
        print(a)
        a += 1
        mae_list_train_sarima.append(mae_sarima_train)
        mae_list_aic_sarima.append(sarima_fit.aic)

        # Forecast using the fitted model
        forecast = sarima_fit.forecast(steps=testing_lenght)

        # Evaluate forecast against actual values
        mae_sarima_test = np.mean(np.abs(test_data['Count'] - forecast))
        mae_list_test_sarima.append(mae_sarima_test)
        print('mae_sarima_test', mae_sarima_test)
        # Generate periodogram
        freq, Pxx_spec = signal.periodogram(
            endog_data["Count"][
                : (train_index+testing_lenght)
            ].values,
            scaling="spectrum",
        )

        maxPsd, freqMax_week, periodMax = getMaxPeriodogram(freq, Pxx_spec, 5)

        frequency_array = np.round(1 / freqMax_week).astype(int)
        period01, period02, period03, period04, period05 = frequency_array
        
        time = np.arange(0, (train_index+testing_lenght))

        # Create regressors
        omega01 = 2 * np.pi / period01
        s1 = np.cos(omega01 * time)
        c1 = np.sin(omega01 * time)

        # Create regressors
        omega02 = 2 * np.pi / period02
        s2 = np.cos(omega02 * time)
        c2 = np.sin(omega02 * time)

        # Create regressors
        omega03 = 2 * np.pi / period03
        s3 = np.cos(omega03 * time)
        c3 = np.sin(omega03 * time)

        # Create regressors
        omega04 = 2 * np.pi / period04
        s4 = np.cos(omega04 * time)
        c4 = np.sin(omega04 * time)

        # Create regressors
        omega05 = 2 * np.pi / period05
        s5 = np.cos(omega05 * time)
        c5 = np.sin(omega05 * time)

        order = (24, 0, 0)

        regressors = np.column_stack((s1, c1, s2, c2, s3, c3, s4, c4, s5, c5))
        try:
            model = SARIMAX(
                endog=train_data,
                exog=regressors[: -testing_lenght],
                trend="c",
                order=order,
                enforce_stationarity=True,
                enforce_invertibility=True,
            )
            sarimax_model = model.fit()
            mae_list_aic_sarimax.append(sarimax_model.aic)
            mae_sarimax_train = np.mean(
                np.abs(train_data['Count'] - sarimax_model.fittedvalues)
            )
            mae_list_train_sarimax.append(mae_sarimax_train)

            # Forecasting
            forecast = sarimax_model.forecast(
                steps=testing_lenght, exog=regressors[-testing_lenght:]
            )
            # Evaluate forecast against actual values
            mae_sarimax_test = np.mean(np.abs(test_data['Count'] - forecast))
            mae_list_test_sarimax.append(mae_sarimax_test)
            print('mae_sarimax_test', mae_sarimax_test)
        except np.linalg.LinAlgError as linalg_err:
            # Handle the LinAlgError here
            print("Caught a LinAlgError as an exception:", linalg_err)
            mae_list_test_sarimax.append("error")
            mae_list_aic_sarimax.append("error")
            mae_list_train_sarimax.append("error")

    df_testing = pd.DataFrame(
        {
            "aic_sarima": mae_list_aic_sarima,
            "mae_test_sarima": mae_list_test_sarima,
            "mae_train_sarima": mae_list_train_sarima,
            "mae_train_sarimax": mae_list_train_sarimax,
            "mae_test_sarimax": mae_list_test_sarimax,
            "aic_sarimax": mae_list_aic_sarimax,
        }
    )

    return df_testing

In [212]:
all_results_48 = []

# Iterate through labels in endog_week_dict
for label, endog_data in endog_week_dict.items():
    print('this is the current label', label)
    results = train_test_split(endog_data, 48, 10)
    results['label'] = label
    all_results.append(results)

# Concatenate all DataFrames into one
df_combined_48 = pd.concat(all_results, ignore_index=True)

folder_path = r"C:\Users\Max_G\OneDrive\IUBH\5_Semester\Model_Engeneering\Public_Transport_Forecasting\data\interim\Experiments_Results"
file_name = "df_mae_all_data_all_models_week_48.csv"

full_path = f"{folder_path}/{file_name}"

df_combined_48.to_csv(full_path, index=True)

this is the current label 0
0
mae_sarima_test 441.9594204138166
mae_sarimax_test 385.22909053149556
1
mae_sarima_test 434.0450553572319
mae_sarimax_test 465.29274044205016
2
mae_sarima_test 421.5523777279031
mae_sarimax_test 372.4115021355861
3
mae_sarima_test 423.26225251636487
mae_sarimax_test 435.72729642063285
4
mae_sarima_test 398.4571889007023
mae_sarimax_test 395.23383497964863
5
mae_sarima_test 465.3094686720631
mae_sarimax_test 471.61911563896706
6
mae_sarima_test 441.80149436207154
mae_sarimax_test 471.7465598510602
7
mae_sarima_test 395.51401505166865
mae_sarimax_test 329.19010140669536
8
mae_sarima_test 391.4425977431551
mae_sarimax_test 398.09783322978126
9
mae_sarima_test 374.24530082085374
mae_sarimax_test 379.5654843695745
this is the current label 1
0
mae_sarima_test 52.681484516561454


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


mae_sarimax_test 53.09910793857526
1
mae_sarima_test 53.53823852807048


c:\Users\Max_G\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


mae_sarimax_test 50.864145991623445


KeyboardInterrupt: 